In [ ]:
# Image Data Augmention (이미지 데이터 증강)

In [ ]:
# 이미지 증강 + 시각화
# - flow()

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# 1. 이미지 증강기 정의
imgGenerator = ImageDataGenerator(
  rescale=1.0 / 255.0, # RGB값 정규화
  rotation_range=30, # 각도 최대 30도 회전
  width_shift_range=0.1, # 좌우로 최대 10% 이동
  height_shift_range=0.1, # 상하로 최대 10% 이동
  shear_range=0.2, # 비스듬히 기울이기
  zoom_range=0.1, # 확대, 축소 최대10%
  horizontal_flip=True, # 50% 확률로 좌우 반전
  fill_mode='nearest' # 회전 시 생기는 빈공간 채우기
)

# 2. 예제 이미지 불러오기
# 이미지 한 장을 불러와서 증강 테스트를 수행
img = load_img('image/a.jpg')
x = img_to_array(img)
x = np.expand_dims(x, axis=0) # 차원 맞추기 (샘플 수)

# 3. 증강된 이미지 시각화
# 증강된 이미지를 2행 5열(총 10장)으로 출력
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()
i = 0

# .flow() 는 넘파이 배열을 기반으로 증강된 이미지 배치를 생성
for batch in imgGenerator.flow(x, batch_size=1):
  augmented_image = batch[0] # 증강된 첫번째 이미지 추출
  
  axes[i].imshow(augmented_image) # 이미지 출력
  axes[i].axis('off')
  axes[i].set_title(f"Aug {i+1}")
  
  i += 1
  
  if i >= 10:
    break
# 레이아웃 조정 및 출력
plt.tight_layout()
plt.show()

In [ ]:
# 이미지 증강 + 시각화 + 증강된 이미지 저장
# - flow()

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

# 0. 저장 폴더 설정
save_dir = 'augmented_images'
os.makedirs(save_dir, exist_ok=True)

# 1. 이미지 증강기 정의
imgGenerator = ImageDataGenerator(
  rescale=1.0 / 255.0, # RGB값 정규화
  rotation_range=30, # 각도 최대 30도 회전
  width_shift_range=0.1, # 좌우로 최대 10% 이동
  height_shift_range=0.1, # 상하로 최대 10% 이동
  shear_range=0.2, # 비스듬히 기울이기
  zoom_range=0.1, # 확대, 축소 최대10%
  horizontal_flip=True, # 50% 확률로 좌우 반전
  fill_mode='nearest' # 회전 시 생기는 빈공간 채우기
)

# 2. 예제 이미지 불러오기
# 이미지 한 장을 불러와서 증강 테스트를 수행
img = load_img('image/a.jpg')
x = img_to_array(img)
x = np.expand_dims(x, axis=0) # 차원 맞추기 (샘플 수)

# 3. 증강된 이미지 시각화
# 증강된 이미지를 2행 5열(총 10장)으로 출력
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()
i = 0

# .flow() 는 넘파이 배열을 기반으로 증강된 이미지 배치를 생성
for batch in imgGenerator.flow(x, batch_size=1):
  augmented_image = batch[0] # 증강된 첫번째 이미지 추출
  image = array_to_img(augmented_image)
  
  # 저장 경로 지정
  filename = f"aug_{i+1:02d}.png"
  image.save(os.path.join(save_dir, filename))
  
  axes[i].imshow(image) # 이미지 출력
  axes[i].axis('off')
  axes[i].set_title(f"Aug {i+1}")
  
  i += 1
  
  if i >= 10:
    break
# 레이아웃 조정 및 출력
plt.tight_layout()
plt.show()

In [ ]:
# 이미지 증강 + 시각화 + 증강된 이미지 저장
# - flow_from_directory()

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img

# 1. 증강 결과 저장 경로 설정
output_base_dir = 'save'
os.makedirs(output_base_dir, exist_ok=True)

# 2. 증강기 정의 (CNN에 적합한 설정)
imgGenerator = ImageDataGenerator(
  rescale=1./255,
  rotation_range=30,
	width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
	fill_mode='nearest'
)

# 3. flow_from_directory: 폴더 구조에서 이미지 불러오기
generator = imgGenerator.flow_from_directory(
  directory='dataset', # 입력 디렉토리
  target_size=(150, 150), # CNN 입력 크기
  batch_size=1, # 한 장씩 처리
  class_mode='categorical', # 다중 클래스 분류
  shuffle=False # 고정된 순서
)

# 클래스 이름 매핑
class_indices = generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# 4. 증강 + 저장 + 시각화
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

saved_count = {className: 0 for className in class_indices.keys()}

i = 0

for x_batch, y_batch in generator:
  # 현재 클래스 이름 얻기
  class_index = np.argmax(y_batch[0])
  class_name = index_to_class[class_index]
  
  # 저장 디렉토리 생성
  save_dir = os.path.join(output_base_dir, class_name)
  os.makedirs(save_dir, exist_ok=True)
  
  # 이미지 변환 및 저장
  img = array_to_img(x_batch[0])
  count = saved_count[class_name]
  filename = f"{class_name}_{count:02d}.jpg"
  img_path = os.path.join(save_dir, filename)
  img.save(img_path)
  saved_count[class_name] += 1
  
  # 시각화용 저장
  if count <= 5:
    axes[i].imshow(img)
    axes[i].axis('off')
    axes[i].set_title(f"{class_name} {count}")
    i += 1
  
  # 클래스당 5장씩 저장되었으면 종료
  if all(v >= 5 for v in saved_count.values()):
    break

plt.tight_layout()
plt.show()

In [ ]:
# DataSet - Cat and Dog (Kaggle): PC작업용

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import shutil

source_filename = "dataset/archive.zip"
extract_folder = "dataset/"
